In [22]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import time

def validate_matrix_dims(mat1, mat2):
    """Проверка, что умножение матриц возможно."""
    if mat1.shape[1] != mat2.shape[0]:
        raise ValueError("Размеры матриц не соответствуют правилам умножения")

def cpu_matrix_mult(mat1, mat2):
    """Умножение матриц на CPU."""
    validate_matrix_dims(mat1, mat2)

    output = np.zeros((mat1.shape[0], mat2.shape[1]))

    start = time.time()
    for row in range(mat1.shape[0]):
        for col in range(mat2.shape[1]):
            for index in range(mat1.shape[1]):
                output[row, col] += mat1[row, index] * mat2[index, col]
    end = time.time()

    return output, end - start

def gpu_matrix_mult(mat1, mat2):
    """Умножение матриц на GPU с использованием PyCUDA."""
    validate_matrix_dims(mat1, mat2)

    # CUDA Kernel
    mod = SourceModule("""
    __global__ void matrixMulKernel(float *A, float *B, float *C, int N) {
        int row = blockIdx.y * blockDim.y + threadIdx.y;
        int col = blockIdx.x * blockDim.x + threadIdx.x;

        if(row < N && col < N) {
            float sum = 0.0;
            for (int k = 0; k < N; k++) {
                sum += A[row * N + k] * B[k * N + col];
            }
            C[row * N + col] = sum;
        }
    }
    """)

    # Размер матриц
    N = mat1.shape[0]

    # Аллокация памяти на GPU
    mat1_gpu = cuda.mem_alloc(mat1.nbytes)
    mat2_gpu = cuda.mem_alloc(mat2.nbytes)
    result_gpu = cuda.mem_alloc(mat1.nbytes)  # результат такой же размерности, что и мат1 и мат2

    # Копирование данных из CPU в GPU
    cuda.memcpy_htod(mat1_gpu, mat1)
    cuda.memcpy_htod(mat2_gpu, mat2)

    # Вызываем CUDA Kernel
    matrix_mul = mod.get_function("matrixMulKernel")

    # Настройка сетки и блоков
    block_size = (16, 16, 1)
    grid_size = (int(np.ceil(N / 16)), int(np.ceil(N / 16)), 1)

    start = time.time()
    matrix_mul(mat1_gpu, mat2_gpu, result_gpu, np.int32(N), block=block_size, grid=grid_size)
    cuda.Context.synchronize()
    end = time.time()

    # Копирование результата с GPU в CPU
    result = np.empty_like(mat1)
    cuda.memcpy_dtoh(result, result_gpu)

    return result, end - start

if __name__ == "__main__":
    shape = (150, 150)
    mat1 = np.random.random(shape).astype(np.float32)
    mat2 = np.random.random(shape).astype(np.float32)

    cpu_result, cpu_exec_time = cpu_matrix_mult(mat1, mat2)
    gpu_result, gpu_exec_time = gpu_matrix_mult(mat1, mat2)

    # print(f"Результат на CPU:\n{cpu_result}")
    print(f"Время выполнения на CPU: {cpu_exec_time:.6f} сек")

    # print(f"Результат на GPU:\n{gpu_result}")
    print(f"Время выполнения на GPU: {gpu_exec_time:.6f} сек")

Время выполнения на CPU: 1.978855 сек
Время выполнения на GPU: 0.001034 сек
